In [9]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.mrtrix.convert as mtxc
import nipype.interfaces.mrtrix.preprocess as mtxp
import nipype.interfaces.fsl as fsl
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
import sys

var1 = str(sys.argv[1])
today = str(date.today())
config.enable_debug_mode()

In [3]:
# Resources for MRTRIX:
# https://community.mrtrix.org/t/the-output-of-tck2connectome/345/25

## Error: FSLDIR not set
# Set module load and sourcing scripts within the activate.d folder file

In [10]:
#Set user and path variables
local='False'
user = expanduser('~')
if user == '/Users/lucindasisk':
    if local == 'True':
        laptop = '/Users/lucindasisk/Desktop/DATA'
        home = join(user, 'Desktop/Milgram/candlab')
        raw_dir = join(home, 'data/mri/bids_recon/shapes')
        proc_dir = join(home, 'analyses/shapes/dwi')
        workflow_dir = join(laptop, 'workflows_ls')
        data_dir = join(laptop, 'data_ls')
    else:
        home = join(user, 'Desktop/Milgram/candlab')
        raw_dir = join(home, 'data/mri/bids_recon/shapes')
        proc_dir = join(home, 'analyses/shapes/dwi/data')
        workflow_dir = join(home, 'analyses/shapes/dwi/workflows')
        data_dir = join(home, 'analyses/shapes/dwi/data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/data')
    workflow_dir = join(home, 'analyses/shapes/dwi/workflows')
    data_dir = join(home, 'analyses/shapes/dwi/data')
    
# Read in subject subject_list
# subject_info = read_csv(
#     home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
# subjects = subject_info[0].tolist()
subject_list = [var1]


In [4]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected.nii.gz'),
                bval = join(raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected.eddy_rotated_bvecs'),
                t1 = join(proc_dir, '1_Check_Unwarped/{subject_id}/{subject_id}_ses-shapesV1_T1w_flirt_brain.nii.gz'),
                mni=join(home, 'atlases/MNI152_T1_2mm_brain.nii.gz')
               )

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

### Nodes for Diffusion workflow

In [5]:
# #Generate binary mask
# bet=Node(fsl.BET(frac=0.2,
#                 mask=True),
#         name='bet')

#Convert bvals and bvecs to fslgrad
gradconv = Node(mtx.MRConvert(),
               name = 'gradconv')

#Generate 5 tissue type (5tt) segmentation using FAST algorithm
seg5tt = Node(mtx.Generate5tt(algorithm = 'fsl',
                             out_file = 'T1s_5tt_segmented.nii.gz'),
             name='seg5tt')

#Estimate response functions for spherical deconvolution using the specified algorithm (Dhollander)
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/preprocess.html#responsesd
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/response_function_estimation.html#response-function-estimation
#Max_sh (lmax variable) determined in shell order from here: https://mrtrix.readthedocs.io/en/3.0_rc2/constrained_spherical_deconvolution/lmax.html
#DWI has 5 shells: 7 b0 volumes, 6 b500 vols, 15 b1000 vols, 15 b2000 bols, 60 b3000 vols
dwiresp = Node(mtx.ResponseSD(algorithm = 'dhollander',
                              max_sh=[0,2,4,4,8],
                              wm_file = 'wm_response.txt',
                              gm_file = 'gm_response.txt',
                              csf_file = 'csf_response.txt'),
              name='dwiresp')

#Estimate fiber orientation distributions from diffusion data sing spherical deconvolution
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/reconst.html
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/multi_shell_multi_tissue_csd.html
#Max SH here determined by tissue type - chose 8,8,8 per forum recommendations
mscsd = Node(mtx.EstimateFOD(algorithm = 'msmt_csd',
                             bval_scale = 'yes',
                            max_sh = [8,8,8]),
            name='mscsd')

#Perform Tractography - iFOD2 (https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/tracking.html) 
tract = Node(mtx.Tractography(algorithm='iFOD2',
                              select=100000, #Jiook has done 100 million streamlines
                              n_trials=10000, 
                              out_file='msCSD_brain_tracktography.tck'),
            name='tract')

In [ ]:
csd_flow = Workflow(name = 'tract_flow')
csd_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                    #Segment T1 image with FSL 5tt algorithm
                    (sf, seg5tt, [('t1', 'in_file')]),
                    (seg5tt, datasink, [('out_file', '5_Tract_Reconstruction')]),
                    #Convert bval/bvec to gradient tables
                    (sf, gradconv, [('dti', 'in_file'),
                                   ('bval','in_bval'),
                                   ('bvec', 'in_bvec')]),
                    #Compute FOD response functions
                    (gradconv, dwiresp, [('out_file', 'in_file')]),
                    (dwiresp, datasink, [('wm_file', '4_Deconvolution.@par'),
                                        ('gm_file', '4_Deconvolution.@par.@par'),
                                        ('csf_file', '4_Deconvolution.@par.@par.@par')]),
                    (gradconv, mscsd, [('out_file', 'in_file')]),
                    #Perform multi-shell constrained spherical deconvolution
                    (dwiresp, mscsd, [('wm_file', 'wm_txt'),
                                      ('gm_file', 'gm_txt'),
                                      ('csf_file', 'csf_txt')]),
                    (mscsd, tract, [('wm_odf', 'in_file')]),
                    (mscsd, datasink, [('wm_odf', '4_Deconvolution.@par.@par.@par.@par'),
                                       ('gm_odf', '4_Deconvolution.@par.@par.@par.@par.@par'),
                                       ('csf_odf','4_Deconvolution.@par.@par.@par.@par.@par.@par')]),
                    (gradconv, datasink, [('out_file', '4_Deconvolution.@par.@par.@par.@par.@par.@par.@par')])
                   ])
csd_flow.base_dir = workflow_dir
csd_flow.write_graph(graph2use = 'flat')
dwi = csd_flow.run('MultiProc', plugin_args={'n_procs': 4})

210311-15:40:17,303 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): No edge data
210311-15:40:17,304 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
210311-15:40:17,306 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): No edge data
210311-15:40:17,307 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): new edge data: {'connect': [('t1', 'in_file')]}
210311-15:40:17,308 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): No edge data
210311-15:40:17,309 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): new edge data: {'connect': [('out_file', '5_Tract_Reconstruction')]}
210311-15:40:17,310 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.gradconv): No edge data
210311-15:40:17,311 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.gradconv): new edge data: {'connect': [('dti', 'in_file'), ('bval', 'in_bval'), ('bvec', 'in_bvec')]}
210311-15

210311-15:40:18,785 nipype.workflow DEBUG:
	 [Node] Hashes: [('base_directory', '/gpfs/milgram/project/gee_dylan/candlab'), ('force_lists', False), ('raise_on_empty', True), ('sort_filelist', True), ('subject_id', 'sub-A200')], 9868cc5e94fad4f0aa2c74f928d06cae, /gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/sf/_0x9868cc5e94fad4f0aa2c74f928d06cae.json, ['/gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/sf/_0x9868cc5e94fad4f0aa2c74f928d06cae.json']
210311-15:40:18,786 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "tract_flow.sf".
210311-15:40:18,789 nipype.workflow DEBUG:
	 [Node] Rerunning cached, up-to-date node "tract_flow.sf"
210311-15:40:18,791 nipype.utils DEBUG:
	 Removing contents of /gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/sf
210311-15:40:18,798 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/gpfs/milg

210311-15:40:20,875 nipype.workflow INFO:
	 [Node] Running "seg5tt" ("nipype.interfaces.mrtrix3.utils.Generate5tt"), a CommandLine Interface with command:
5ttgen fsl /gpfs/milgram/project/gee_dylan/candlab/analyses/shapes/dwi/data/1_Check_Unwarped/sub-A200/sub-A200_ses-shapesV1_T1w_flirt_brain.nii.gz T1s_5tt_segmented.nii.gz
210311-15:40:20,880 nipype.interface DEBUG:
	 algorithm_fsl
210311-15:40:20,883 nipype.interface DEBUG:
	 in_file_/gpfs/milgram/project/gee_dylan/candlab/analyses/shapes/dwi/data/1_Check_Unwarped/sub-A200/sub-A200_ses-shapesV1_T1w_flirt_brain.nii.gz
210311-15:40:20,885 nipype.interface DEBUG:
	 out_file_T1s_5tt_segmented.nii.gz
210311-15:40:22,638 nipype.workflow DEBUG:
	 Progress: 7 jobs, 2/1/1 (done/running/ready), 1/4 (pending_tasks/waiting).
210311-15:40:22,639 nipype.workflow DEBUG:
	 Tasks currently running: 1. Pending: 1.
210311-15:40:22,642 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 225.91/226.11, Free processor

210311-15:40:24,911 nipype.interface DEBUG:
	 gm_odf_gm.mif
210311-15:40:24,912 nipype.interface DEBUG:
	 gm_txt_/gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt
210311-15:40:24,921 nipype.interface DEBUG:
	 in_file_/gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/gradconv/dwi.mif
210311-15:40:24,922 nipype.interface DEBUG:
	 max_sh_[8, 8, 8]
210311-15:40:24,927 nipype.interface DEBUG:
	 wm_odf_wm.mif
210311-15:40:24,929 nipype.interface DEBUG:
	 wm_txt_/gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/dwi/workflows/tract_flow/_subject_id_sub-A200/dwiresp/wm_response.txt
210311-15:40:26,640 nipype.workflow DEBUG:
	 Progress: 7 jobs, 3/2/0 (done/running/ready), 2/2 (pending_tasks/waiting).
210311-15:40:26,641 nipype.workflow DEBUG:
	 Tasks currently running: 2. Pending: 2.
210311-15:40:26,642 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free 